# Trabalho Prático 2 - Planejamento e Navegação
## Passo 1b: Roadmap para Pioneer p3dx

### Importação de Bibliotecas

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import networkx as nx
from matplotlib.patches import Circle, Rectangle
from scipy.spatial import distance

### Definição dos Parâmetros do Robô

O Pioneer p3dx é um robô diferencial (não-holonômico) com as seguintes características:
- Raio aproximado: 0.25 m
- Restrições de movimento: não pode se mover lateralmente
- Raio mínimo de curvatura devido às restrições cinemáticas

In [ ]:
# Parâmetros do Pioneer p3dx
ROBOT_RADIUS = 0.25  # raio do robô em metros
SAFETY_MARGIN = 0.15  # margem de segurança adicional
EFFECTIVE_RADIUS = ROBOT_RADIUS + SAFETY_MARGIN
MIN_TURNING_RADIUS = 0.5  # raio mínimo de curvatura em metros

### Carregamento dos Mapas

In [ ]:
# Carregar mapas
mapa1 = mpimg.imread('imagens/mapa1_invertido.png')
mapa2 = mpimg.imread('imagens/mapa2_invertido.png')

# Dimensões dos mapas
WORLD_WIDTH = 50  # largura em metros
WORLD_HEIGHT = 30  # altura em metros

### Implementação do Algoritmo Roadmap para Robô Diferencial

Para o Pioneer p3dx, precisamos considerar as restrições cinemáticas do robô diferencial.
O algoritmo Roadmap é adaptado para considerar:
1. Maior margem de segurança devido às limitações de manobra
2. Verificação de viabilidade considerando o raio mínimo de curvatura
3. Orientação do robô ao longo do caminho

In [ ]:
def pixel_to_world(px, py, img_shape, world_width, world_height):
    """Converte coordenadas de pixel para coordenadas do mundo."""
    h, w = img_shape[:2]
    x = (px / w) * world_width
    y = (py / h) * world_height
    return x, y

def world_to_pixel(x, y, img_shape, world_width, world_height):
    """Converte coordenadas do mundo para coordenadas de pixel."""
    h, w = img_shape[:2]
    px = int((x / world_width) * w)
    py = int((y / world_height) * h)
    return px, py

def is_collision_free(x1, y1, x2, y2, mapa, robot_radius, world_width, world_height):
    """Verifica se o caminho entre dois pontos está livre de colisões."""
    num_checks = int(distance.euclidean([x1, y1], [x2, y2]) * 10)
    num_checks = max(num_checks, 10)
    
    for i in range(num_checks + 1):
        t = i / num_checks
        x = x1 + t * (x2 - x1)
        y = y1 + t * (y2 - y1)
        
        px, py = world_to_pixel(x, y, mapa.shape, world_width, world_height)
        
        # Verificar área ao redor do ponto considerando o raio do robô
        radius_px = int((robot_radius / world_width) * mapa.shape[1])
        
        for dx in range(-radius_px, radius_px + 1):
            for dy in range(-radius_px, radius_px + 1):
                if dx*dx + dy*dy <= radius_px*radius_px:
                    check_px = px + dx
                    check_py = py + dy
                    
                    if 0 <= check_px < mapa.shape[1] and 0 <= check_py < mapa.shape[0]:
                        if mapa[check_py, check_px, 0] < 0.5:  # obstáculo
                            return False
    return True

def check_turning_radius(x1, y1, x2, y2, x3, y3, min_radius):
    """Verifica se o raio de curvatura entre três pontos consecutivos é viável."""
    # Calcular o raio do círculo que passa pelos três pontos
    # Se os pontos são colineares ou o raio é grande o suficiente, retorna True
    
    # Vetores
    v1 = np.array([x2 - x1, y2 - y1])
    v2 = np.array([x3 - x2, y3 - y2])
    
    # Se os vetores são quase paralelos, não há problema
    cross = np.cross(v1, v2)
    if abs(cross) < 0.01:
        return True
    
    # Calcular raio aproximado
    angle = np.arccos(np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1))
    
    # Se o ângulo é pequeno, a curva é suave
    if angle < np.pi / 6:  # menos de 30 graus
        return True
    
    # Estimativa conservadora do raio
    chord = distance.euclidean([x1, y1], [x3, y3])
    radius_estimate = chord / (2 * np.sin(angle / 2)) if angle > 0.01 else float('inf')
    
    return radius_estimate >= min_radius

def generate_random_samples(mapa, num_samples, robot_radius, world_width, world_height):
    """Gera amostras aleatórias no espaço livre."""
    samples = []
    attempts = 0
    max_attempts = num_samples * 100
    
    while len(samples) < num_samples and attempts < max_attempts:
        x = np.random.uniform(0, world_width)
        y = np.random.uniform(0, world_height)
        
        px, py = world_to_pixel(x, y, mapa.shape, world_width, world_height)
        
        # Verificar se o ponto está livre
        radius_px = int((robot_radius / world_width) * mapa.shape[1])
        is_free = True
        
        for dx in range(-radius_px, radius_px + 1):
            for dy in range(-radius_px, radius_px + 1):
                if dx*dx + dy*dy <= radius_px*radius_px:
                    check_px = px + dx
                    check_py = py + dy
                    
                    if 0 <= check_px < mapa.shape[1] and 0 <= check_py < mapa.shape[0]:
                        if mapa[check_py, check_px, 0] < 0.5:
                            is_free = False
                            break
            if not is_free:
                break
        
        if is_free:
            samples.append((x, y))
        
        attempts += 1
    
    return samples

def build_roadmap(samples, start, goal, mapa, robot_radius, world_width, world_height, k_nearest=10):
    """Constrói o grafo roadmap conectando amostras próximas."""
    G = nx.Graph()
    
    # Adicionar todos os pontos incluindo início e objetivo
    all_points = [start] + samples + [goal]
    
    for i, point in enumerate(all_points):
        G.add_node(i, pos=point)
    
    # Conectar cada ponto aos k vizinhos mais próximos
    for i, p1 in enumerate(all_points):
        distances = []
        for j, p2 in enumerate(all_points):
            if i != j:
                dist = distance.euclidean(p1, p2)
                distances.append((j, dist))
        
        # Ordenar por distância e pegar os k mais próximos
        distances.sort(key=lambda x: x[1])
        
        for j, dist in distances[:k_nearest]:
            p2 = all_points[j]
            if is_collision_free(p1[0], p1[1], p2[0], p2[1], mapa, robot_radius, world_width, world_height):
                G.add_edge(i, j, weight=dist)
    
    return G, all_points

def find_path(G, start_idx, goal_idx):
    """Encontra o caminho mais curto no grafo usando A*."""
    try:
        path_indices = nx.astar_path(G, start_idx, goal_idx, weight='weight')
        return path_indices
    except nx.NetworkXNoPath:
        return None

def smooth_path_for_differential(path, min_radius):
    """Suaviza o caminho removendo pontos que criam curvas muito acentuadas."""
    if len(path) <= 2:
        return path
    
    smoothed = [path[0]]
    
    i = 1
    while i < len(path) - 1:
        # Verificar se podemos pular este ponto
        if i + 1 < len(path):
            x1, y1 = smoothed[-1]
            x2, y2 = path[i]
            x3, y3 = path[i + 1]
            
            if check_turning_radius(x1, y1, x2, y2, x3, y3, min_radius):
                smoothed.append(path[i])
        i += 1
    
    smoothed.append(path[-1])
    return smoothed

### Experimento 1: Mapa 1 - Cenário Simples

In [ ]:
# Definir pontos de início e objetivo para Mapa 1
start_mapa1 = (5, 15)
goal_mapa1 = (45, 15)

# Gerar amostras
num_samples = 250
samples_mapa1 = generate_random_samples(mapa1, num_samples, EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT)

print(f"Amostras geradas: {len(samples_mapa1)}")

# Construir roadmap
G_mapa1, all_points_mapa1 = build_roadmap(samples_mapa1, start_mapa1, goal_mapa1, mapa1, 
                                           EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT, k_nearest=12)

print(f"Grafo construído com {G_mapa1.number_of_nodes()} nós e {G_mapa1.number_of_edges()} arestas")

# Encontrar caminho
path_indices_mapa1 = find_path(G_mapa1, 0, len(all_points_mapa1) - 1)

if path_indices_mapa1:
    path_mapa1 = [all_points_mapa1[i] for i in path_indices_mapa1]
    
    # Suavizar caminho para robô diferencial
    path_mapa1_smooth = smooth_path_for_differential(path_mapa1, MIN_TURNING_RADIUS)
    
    path_length_mapa1 = sum([distance.euclidean(path_mapa1_smooth[i], path_mapa1_smooth[i+1]) 
                              for i in range(len(path_mapa1_smooth)-1)])
    print(f"Caminho encontrado com comprimento: {path_length_mapa1:.2f} m")
    print(f"Número de waypoints após suavização: {len(path_mapa1_smooth)}")
else:
    print("Nenhum caminho encontrado")

### Visualização do Resultado - Mapa 1

In [ ]:
if path_indices_mapa1:
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(mapa1, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')
    
    # Desenhar grafo
    for edge in G_mapa1.edges():
        p1 = all_points_mapa1[edge[0]]
        p2 = all_points_mapa1[edge[1]]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', alpha=0.2, linewidth=0.5)
    
    # Desenhar amostras
    for point in samples_mapa1:
        ax.plot(point[0], point[1], 'bo', markersize=2)
    
    # Desenhar caminho original
    path_x = [p[0] for p in path_mapa1]
    path_y = [p[1] for p in path_mapa1]
    ax.plot(path_x, path_y, 'y-', linewidth=2, alpha=0.5)
    
    # Desenhar caminho suavizado
    smooth_x = [p[0] for p in path_mapa1_smooth]
    smooth_y = [p[1] for p in path_mapa1_smooth]
    ax.plot(smooth_x, smooth_y, 'r-', linewidth=3)
    
    # Desenhar início e objetivo
    ax.plot(start_mapa1[0], start_mapa1[1], 'go', markersize=10)
    ax.plot(goal_mapa1[0], goal_mapa1[1], 'ro', markersize=10)
    
    # Desenhar orientação do robô em alguns pontos
    for i in range(0, len(path_mapa1_smooth)-1, max(1, len(path_mapa1_smooth)//10)):
        x, y = path_mapa1_smooth[i]
        if i < len(path_mapa1_smooth) - 1:
            dx = path_mapa1_smooth[i+1][0] - x
            dy = path_mapa1_smooth[i+1][1] - y
            ax.arrow(x, y, dx*0.3, dy*0.3, head_width=0.5, head_length=0.3, fc='orange', ec='orange')
    
    ax.set_xlim(0, WORLD_WIDTH)
    ax.set_ylim(0, WORLD_HEIGHT)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig('imagens/roadmap_mapa1_pioneer.png', dpi=150, bbox_inches='tight')
    plt.show()

### Experimento 2: Mapa 2 - Cenário Complexo

In [ ]:
# Definir pontos de início e objetivo para Mapa 2
start_mapa2 = (5, 5)
goal_mapa2 = (45, 25)

# Gerar mais amostras para o mapa mais complexo
num_samples_2 = 350
samples_mapa2 = generate_random_samples(mapa2, num_samples_2, EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT)

print(f"Amostras geradas: {len(samples_mapa2)}")

# Construir roadmap
G_mapa2, all_points_mapa2 = build_roadmap(samples_mapa2, start_mapa2, goal_mapa2, mapa2, 
                                           EFFECTIVE_RADIUS, WORLD_WIDTH, WORLD_HEIGHT, k_nearest=12)

print(f"Grafo construído com {G_mapa2.number_of_nodes()} nós e {G_mapa2.number_of_edges()} arestas")

# Encontrar caminho
path_indices_mapa2 = find_path(G_mapa2, 0, len(all_points_mapa2) - 1)

if path_indices_mapa2:
    path_mapa2 = [all_points_mapa2[i] for i in path_indices_mapa2]
    
    # Suavizar caminho para robô diferencial
    path_mapa2_smooth = smooth_path_for_differential(path_mapa2, MIN_TURNING_RADIUS)
    
    path_length_mapa2 = sum([distance.euclidean(path_mapa2_smooth[i], path_mapa2_smooth[i+1]) 
                              for i in range(len(path_mapa2_smooth)-1)])
    print(f"Caminho encontrado com comprimento: {path_length_mapa2:.2f} m")
    print(f"Número de waypoints após suavização: {len(path_mapa2_smooth)}")
else:
    print("Nenhum caminho encontrado")

### Visualização do Resultado - Mapa 2

In [ ]:
if path_indices_mapa2:
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(mapa2, extent=[0, WORLD_WIDTH, 0, WORLD_HEIGHT], origin='lower')
    
    # Desenhar grafo
    for edge in G_mapa2.edges():
        p1 = all_points_mapa2[edge[0]]
        p2 = all_points_mapa2[edge[1]]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], 'b-', alpha=0.2, linewidth=0.5)
    
    # Desenhar amostras
    for point in samples_mapa2:
        ax.plot(point[0], point[1], 'bo', markersize=2)
    
    # Desenhar caminho original
    path_x = [p[0] for p in path_mapa2]
    path_y = [p[1] for p in path_mapa2]
    ax.plot(path_x, path_y, 'y-', linewidth=2, alpha=0.5)
    
    # Desenhar caminho suavizado
    smooth_x = [p[0] for p in path_mapa2_smooth]
    smooth_y = [p[1] for p in path_mapa2_smooth]
    ax.plot(smooth_x, smooth_y, 'r-', linewidth=3)
    
    # Desenhar início e objetivo
    ax.plot(start_mapa2[0], start_mapa2[1], 'go', markersize=10)
    ax.plot(goal_mapa2[0], goal_mapa2[1], 'ro', markersize=10)
    
    # Desenhar orientação do robô em alguns pontos
    for i in range(0, len(path_mapa2_smooth)-1, max(1, len(path_mapa2_smooth)//10)):
        x, y = path_mapa2_smooth[i]
        if i < len(path_mapa2_smooth) - 1:
            dx = path_mapa2_smooth[i+1][0] - x
            dy = path_mapa2_smooth[i+1][1] - y
            ax.arrow(x, y, dx*0.3, dy*0.3, head_width=0.5, head_length=0.3, fc='orange', ec='orange')
    
    ax.set_xlim(0, WORLD_WIDTH)
    ax.set_ylim(0, WORLD_HEIGHT)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig('imagens/roadmap_mapa2_pioneer.png', dpi=150, bbox_inches='tight')
    plt.show()

### Análise dos Resultados

O algoritmo Roadmap foi adaptado para o Pioneer p3dx considerando suas restrições cinemáticas:

1. **Margem de Segurança Maior**: Devido às limitações de manobra do robô diferencial, utilizamos uma margem de segurança maior
2. **Suavização de Trajetória**: O caminho foi suavizado para respeitar o raio mínimo de curvatura
3. **Orientação do Robô**: As setas indicam a orientação necessária do robô ao longo do caminho
4. **Comparação com Youbot**: O Pioneer p3dx requer caminhos mais suaves e pode ter trajetórias ligeiramente mais longas devido às restrições não-holonômicas

Os resultados mostram que o método é eficaz mesmo considerando as restrições do robô diferencial.